In [1]:
import pandas as pd
import mlflow

from sklearn.model_selection import train_test_split

import json
import requests

In [2]:
# Preprocessing 
df = pd.read_csv("datasets/insurance.csv")
df = df.drop(columns=['region'])
df['sex']=df['sex'].map({'female':0,'male':1}).astype(int)
df['smoker']=df['smoker'].map({'yes':0,'no':1}).astype(int)

In [3]:
# Training Data
X = df[["age", "bmi", "children", "smoker", "charges"]]
y = df[["sex"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [50]:
df.iloc[0]

index           0.000
age            19.000
sex             0.000
bmi            27.900
children        0.000
smoker          0.000
charges     16884.924
Name: 0, dtype: float64

In [62]:
d1 = {'dataframe_split': X_test.iloc[0:1].to_dict(orient='split', index=False)}
# d1 = {'dataframe_split': X_test.iloc[0].to_dict(orient='split')}
d1

{'dataframe_split': {'columns': ['age',
   'bmi',
   'children',
   'smoker',
   'charges'],
  'data': [[45, 25.175, 2, 1, 9095.06825]]}}

In [55]:
dt = json.dumps(d1)
dt

'{"dataframe_split": {"age": 45.0, "bmi": 25.175, "children": 2.0, "smoker": 1.0, "charges": 9095.06825}}'

In [56]:
response = requests.post(url='http://127.0.0.1:8080/invocations', data=dt, headers={"Content-Type": "application/json"})

In [57]:
print(response.json())

{'error_code': 'BAD_REQUEST', 'message': "Invalid input. Dataframe split format must have 'data' field and optionally 'columns' and 'index' fields. Got {'age', 'smoker', 'bmi', 'children', 'charges'}.'"}


In [74]:
d2 = {'dataframe_records':X_test.iloc[5:6].to_dict(orient='records')}
# d2 = {'dataframe_records':X_test.iloc[1].to_dict()}

dt = json.dumps(d2)
response = requests.post(url='http://127.0.0.1:8080/invocations', data=dt, headers={"Content-Type": "application/json"})
print(response.json())

{'predictions': [0]}


## CSV

In [32]:
# c1 = 
X_test.iloc[0]

age           45.00000
bmi           25.17500
children       2.00000
smoker         1.00000
charges     9095.06825
Name: 764, dtype: float64

In [38]:
# X_test.iloc[0].to_csv(header=False, index=False,)
X_test.iloc[0:1].to_csv(header=False, index=False).strip('\n').split('\n')

['45,25.175,2,1,9095.06825\r']

In [57]:
c1 = tuple(X_test.iloc[1].astype(str).values.flatten())
c1

('36.0', '30.02', '0.0', '1.0', '5272.1758')

In [58]:
type(c1)

tuple

In [38]:
# response = requests.post(url='http://127.0.0.1:8080/invocations', data=c1, headers={"Content-Type": "application/csv"})

## Run Service

In [6]:
run_id = 'aa1a55e6db784f8e897de8aba06a47f9'

In [8]:
local_serve_uri = ''

run_id_serve_uri = f"runs:/{run_id}/model"
run_id_serve_uri


'runs:/aa1a55e6db784f8e897de8aba06a47f9/model'

run in bash :

mlflow models serve -m runs:/aa1a55e6db784f8e897de8aba06a47f9/model

mlflow models serve -m file:///D:/Prg/DS/MLFlow/MLFLow_Datacamp/mlruns/0/aa1a55e6db784f8e897de8aba06a47f9/artifacts/model -p 8080 --no-conda

In [ ]:
mlflow models serve -m file:///D:/Prg/DS/MLFlow/MLFLow_Datacamp/mlruns/0/aa1a55e6db784f8e897de8aba06a47f9/artifacts/model \
-p 8080 --no-conda --enable-mlserver

In [47]:
!curl http://127.0.0.1:5000/invocations -H 'Content-Type:application/json' \
-d '{"dataframe_split":{"columns": ["age", "bmi", "children", "smoker", "charges"],\
 "data": [[19, 27.9, 0, 0, 16884.924]]}}'

This predictor only supports the following content types: Types: ['text/csv', 'application/json']. Got 'application/x-www-form-urlencoded'.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   166  100   139  100    27  32116   6238 --:--:-- --:--:-- --:--:-- 41500
curl: (3) bad range in URL position 2:
[age,
 ^


In [50]:
!curl -X POST \
  http://127.0.0.1:50000/invocations \
  -H 'Content-Type: application/json' \
  -d '{"dataframe_split":{"columns": ["age", "bmi", "children", "smoker", "charges"],\
 "data": [[19, 27.9, 0, 0, 16884.924]]}}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
curl: (7) Failed to connect to 127.0.0.1 port 50000 after 2025 ms: Couldn't connect to server
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0curl: (6) Could not resolve host: application
curl: (3) bad range in URL position 2:
[age,
 ^
